<a href="https://colab.research.google.com/github/VideoPac/hello-world/blob/master/SparringGrinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install poker


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 930.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 9.8 MB/s eta 0:00:00


In [2]:
!pip install eval7


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.6/591.6 kB 12.5 MB/s eta 0:00:00


In [4]:
import eval7

# Créer un deck
deck = eval7.Deck()
deck.shuffle()

# Distribuer une main de 7 cartes
hand = deck.deal(7)
print(hand)

# Évaluer la main
hand_value = eval7.evaluate(hand)
hand_type = eval7.handtype(hand_value)
print(f"Valeur de la main : {hand_value}, Type de main : {hand_type}")


[Card("As"), Card("Jh"), Card("7s"), Card("3h"), Card("Qh"), Card("Ks"), Card("8h")]
Valeur de la main : 834198, Type de main : High Card


In [5]:
import eval7
from itertools import combinations
import time

def generate_boards(deck, hand1, hand2):
    hand1_cards = [eval7.Card(c) for c in hand1]
    hand2_cards = [eval7.Card(c) for c in hand2]

    # Enlever les cartes des mains du deck
    for card in hand1_cards + hand2_cards:
        deck.cards.remove(card)

    # Générer toutes les combinaisons de 5 cartes sans les cartes des mains
    board_combinations = list(combinations(deck.cards, 5))

    return board_combinations

def evaluate_boards(hand1, hand2, board_combinations):
    hand1_wins = 0
    hand2_wins = 0
    ties = 0

    hand1_cards = [eval7.Card(c) for c in hand1]
    hand2_cards = [eval7.Card(c) for c in hand2]

    for board in board_combinations:
        hand1_value = eval7.evaluate(hand1_cards + list(board))
        hand2_value = eval7.evaluate(hand2_cards + list(board))

        if hand1_value > hand2_value:
            hand1_wins += 1
        elif hand2_value > hand1_value:
            hand2_wins += 1
        else:
            ties += 1

    return hand1_wins, hand2_wins, ties

def calculate_equity(hand1, hand2):
    start_time = time.time()

    deck = eval7.Deck()
    board_combinations = generate_boards(deck, hand1, hand2)
    hand1_wins, hand2_wins, ties = evaluate_boards(hand1, hand2, board_combinations)

    total = hand1_wins + hand2_wins + ties
    hand1_equity = (hand1_wins + ties / 2) / total * 100
    hand2_equity = (hand2_wins + ties / 2) / total * 100

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time:.4f} seconds")

    return round(hand1_equity, 2), round(hand2_equity, 2)

# Exemple d'utilisation
hand1 = ['Ah', 'As']
hand2 = ['Kh', 'Ks']
equity = calculate_equity(hand1, hand2)
print(f"Équité pour {hand1}: {equity[0]:.2f}%")
print(f"Équité pour {hand2}: {equity[1]:.2f}%")


Execution time: 6.4853 seconds
Équité pour ['Ah', 'As']: 82.64%
Équité pour ['Kh', 'Ks']: 17.36%


In [6]:
import eval7
from itertools import combinations
import pandas as pd

# Créer les en-têtes pour les lignes et les colonnes
ranks = ['AA', 'KK', 'QQ', 'JJ', 'TT', '99', '88', '77', '66', '55', '44', '33', '22']

# Initialiser un DataFrame vide avec les en-têtes appropriées
equity_df = pd.DataFrame(index=ranks, columns=ranks)

# Remplir la diagonale avec None
for i in range(len(ranks)):
    equity_df.iloc[i, i] = None

# Fonction pour générer toutes les combinaisons de boards
def generate_boards(deck, hand1, hand2):
    hand1_cards = [eval7.Card(c) for c in hand1]
    hand2_cards = [eval7.Card(c) for c in hand2]

    for card in hand1_cards + hand2_cards:
        deck.cards.remove(card)

    board_combinations = list(combinations(deck.cards, 5))
    return board_combinations

# Fonction pour évaluer les boards
def evaluate_boards(hand1, hand2, board_combinations):
    hand1_wins = 0
    hand2_wins = 0
    ties = 0

    hand1_cards = [eval7.Card(c) for c in hand1]
    hand2_cards = [eval7.Card(c) for c in hand2]

    for board in board_combinations:
        hand1_value = eval7.evaluate(hand1_cards + list(board))
        hand2_value = eval7.evaluate(hand2_cards + list(board))

        if hand1_value > hand2_value:
            hand1_wins += 1
        elif hand2_value > hand1_value:
            hand2_wins += 1
        else:
            ties += 1

    return hand1_wins, hand2_wins, ties

# Fonction pour calculer l'équité
def calculate_equity(hand1, hand2):
    deck = eval7.Deck()
    board_combinations = generate_boards(deck, hand1, hand2)
    hand1_wins, hand2_wins, ties = evaluate_boards(hand1, hand2, board_combinations)

    total = hand1_wins + hand2_wins + ties
    hand1_equity = (hand1_wins + ties / 2) / total * 100
    hand2_equity = (hand2_wins + ties / 2) / total * 100

    return round(hand1_equity, 2), round(hand2_equity, 2)

# Remplir le tableau avec les équités des matchup de pocket pairs
for i in range(1, len(ranks)):
    for j in range(i):
        hand1 = [ranks[i][0] + 's', ranks[i][0] + 'h']
        hand2 = [ranks[j][0] + 's', ranks[j][0] + 'h']
        equity1, equity2 = calculate_equity(hand1, hand2)
        equity_df.iloc[i, j] = equity1
        equity_df.iloc[j, i] = equity2

# Afficher le DataFrame
print(equity_df)


       AA     KK     QQ     JJ     TT     99     88     77     66     55  \
AA   None  82.64  82.23  81.82  81.41  81.53  81.12  81.14  81.11  81.55   
KK  17.36   None  82.62  82.22  81.81  81.48  81.54  81.13  81.15  81.14   
QQ  17.77  17.38   None  82.66  82.25  81.86  81.54  81.59  81.18  81.17   
JJ  18.18  17.78  17.34   None  82.68  82.29  81.91  81.58  81.63   81.2   
TT  18.59  18.19  17.75  17.32   None  82.69  82.31  81.92   81.6  81.62   
99  18.47  18.52  18.14  17.71  17.31   None  82.63  82.24  81.86   81.5   
88  18.88  18.46  18.46  18.09  17.69  17.37   None  82.51  82.13  81.71   
77  18.86  18.87  18.41  18.42  18.08  17.76  17.49   None  82.33  81.92   
66  18.89  18.85  18.82  18.37   18.4  18.14  17.87  17.67   None  82.05   
55  18.45  18.86  18.83   18.8  18.38   18.5  18.29  18.08  17.95   None   
44   18.0  18.42  18.38  18.36  18.36  18.02  18.19  18.04  17.91  17.83   
33  17.56  17.97  17.94  17.91  17.92   18.0  17.71  17.94  17.87  17.79   
22  17.11  1

In [8]:
import eval7
from itertools import combinations
import pandas as pd

# Créer les en-têtes pour les lignes et les colonnes
ranks = ['AA', 'KK', 'QQ', 'JJ', 'TT', '99', '88', '77', '66', '55', '44', '33', '22']

# Initialiser un DataFrame vide avec les en-têtes appropriées
equity_diff_color_df = pd.DataFrame(index=ranks, columns=ranks)

# Remplir la diagonale avec None
for i in range(len(ranks)):
    equity_diff_color_df.iloc[i, i] = None

# Fonction pour générer toutes les combinaisons de boards
def generate_boards(deck, hand1, hand2):
    hand1_cards = [eval7.Card(c) for c in hand1]
    hand2_cards = [eval7.Card(c) for c in hand2]

    for card in hand1_cards + hand2_cards:
        deck.cards.remove(card)

    board_combinations = list(combinations(deck.cards, 5))
    return board_combinations

# Fonction pour évaluer les boards
def evaluate_boards(hand1, hand2, board_combinations):
    hand1_wins = 0
    hand2_wins = 0
    ties = 0

    hand1_cards = [eval7.Card(c) for c in hand1]
    hand2_cards = [eval7.Card(c) for c in hand2]

    for board in board_combinations:
        hand1_value = eval7.evaluate(hand1_cards + list(board))
        hand2_value = eval7.evaluate(hand2_cards + list(board))

        if hand1_value > hand2_value:
            hand1_wins += 1
        elif hand2_value > hand1_value:
            hand2_wins += 1
        else:
            ties += 1

    return hand1_wins, hand2_wins, ties

# Fonction pour calculer l'équité
def calculate_equity(hand1, hand2):
    deck = eval7.Deck()
    board_combinations = generate_boards(deck, hand1, hand2)
    hand1_wins, hand2_wins, ties = evaluate_boards(hand1, hand2, board_combinations)

    total = hand1_wins + hand2_wins + ties
    hand1_equity = (hand1_wins + ties / 2) / total * 100
    hand2_equity = (hand2_wins + ties / 2) / total * 100

    return round(hand1_equity, 2), round(hand2_equity, 2)

# Remplir le tableau avec les équités des matchup de pocket pairs avec couleurs différentes
for i in range(1, len(ranks)):
    for j in range(i):
        hand1 = [ranks[i][0] + 's', ranks[i][0] + 'h']
        hand2 = [ranks[j][0] + 's', ranks[j][0] + 'd']
        equity1, equity2 = calculate_equity(hand1, hand2)
        equity_diff_color_df.iloc[i, j] = equity1
        equity_diff_color_df.iloc[j, i] = 100 - equity1

# Afficher le DataFrame
print(equity_diff_color_df)


       AA     KK     QQ     JJ     TT     99     88     77     66     55  \
AA   None  81.95  81.55  81.15  80.75  80.86  80.46  80.48  80.46  80.89   
KK  18.05   None  81.93  81.54  81.14  80.82  80.87  80.47  80.49  80.48   
QQ  18.45  18.07   None  81.97  81.57  81.19  80.87  80.92  80.53  80.52   
JJ  18.85  18.46  18.03   None  81.99  81.61  81.23  80.91  80.97  80.54   
TT  19.25  18.86  18.43  18.01   None   82.0  81.62  81.25  80.93  80.96   
99  19.14  19.18  18.81  18.39   18.0   None  81.94  81.56  81.18  80.84   
88  19.54  19.13  19.13  18.77  18.38  18.06   None  81.82  81.44  81.04   
77  19.52  19.53  19.08  19.09  18.75  18.44  18.18   None  81.64  81.24   
66  19.54  19.51  19.47  19.03  19.07  18.82  18.56  18.36   None  81.36   
55  19.11  19.52  19.48  19.46  19.04  19.16  18.96  18.76  18.64   None   
44  18.67  19.08  19.04  19.02  19.02  18.69  18.86  18.72   18.6  18.52   
33  18.23  18.63   18.6  18.58  18.58  18.66  18.38  18.61  18.55  18.48   
22  17.78  1

In [9]:
import eval7
from itertools import combinations
import pandas as pd

# Créer les en-têtes pour les lignes et les colonnes
ranks = ['AA', 'KK', 'QQ', 'JJ', 'TT', '99', '88', '77', '66', '55', '44', '33', '22']

# Initialiser un DataFrame vide avec les en-têtes appropriées
equity_diff_suit_df = pd.DataFrame(index=ranks, columns=ranks)

# Remplir la diagonale avec None
for i in range(len(ranks)):
    equity_diff_suit_df.iloc[i, i] = None

# Fonction pour générer toutes les combinaisons de boards
def generate_boards(deck, hand1, hand2):
    hand1_cards = [eval7.Card(c) for c in hand1]
    hand2_cards = [eval7.Card(c) for c in hand2]

    for card in hand1_cards + hand2_cards:
        deck.cards.remove(card)

    board_combinations = list(combinations(deck.cards, 5))
    return board_combinations

# Fonction pour évaluer les boards
def evaluate_boards(hand1, hand2, board_combinations):
    hand1_wins = 0
    hand2_wins = 0
    ties = 0

    hand1_cards = [eval7.Card(c) for c in hand1]
    hand2_cards = [eval7.Card(c) for c in hand2]

    for board in board_combinations:
        hand1_value = eval7.evaluate(hand1_cards + list(board))
        hand2_value = eval7.evaluate(hand2_cards + list(board))

        if hand1_value > hand2_value:
            hand1_wins += 1
        elif hand2_value > hand1_value:
            hand2_wins += 1
        else:
            ties += 1

    return hand1_wins, hand2_wins, ties

# Fonction pour calculer l'équité
def calculate_equity(hand1, hand2):
    deck = eval7.Deck()
    board_combinations = generate_boards(deck, hand1, hand2)
    hand1_wins, hand2_wins, ties = evaluate_boards(hand1, hand2, board_combinations)

    total = hand1_wins + hand2_wins + ties
    hand1_equity = (hand1_wins + ties / 2) / total * 100
    hand2_equity = (hand2_wins + ties / 2) / total * 100

    return round(hand1_equity, 2), round(hand2_equity, 2)

# Remplir le tableau avec les équités des matchup de pocket pairs avec toutes les couleurs différentes
for i in range(1, len(ranks)):
    for j in range(i):
        hand1 = [ranks[i][0] + 's', ranks[i][0] + 'h']
        hand2 = [ranks[j][0] + 'c', ranks[j][0] + 'd']
        equity1, equity2 = calculate_equity(hand1, hand2)
        equity_diff_suit_df.iloc[i, j] = equity1
        equity_diff_suit_df.iloc[j, i] = 100 - equity1

# Afficher le DataFrame
print(equity_diff_suit_df)


       AA     KK     QQ     JJ     TT     99     88     77     66     55  \
AA   None  81.26  80.86  80.47  80.08  80.19   79.8  79.82   79.8  80.23   
KK  18.74   None  81.24  80.85  80.46  80.15   80.2  79.81  79.84  79.83   
QQ  19.14  18.76   None  81.28  80.89  80.52   80.2  80.26  79.87  79.86   
JJ  19.53  19.15  18.72   None  81.29  80.93  80.56  80.24   80.3  79.88   
TT  19.92  19.54  19.11  18.71   None  81.31  80.94  80.57  80.26  80.29   
99  19.81  19.85  19.48  19.07  18.69   None  81.24  80.88  80.51  80.17   
88   20.2   19.8   19.8  19.44  19.06  18.76   None  81.13  80.76  80.37   
77  20.18  20.19  19.74  19.76  19.43  19.12  18.87   None  80.95  80.56   
66   20.2  20.16  20.13   19.7  19.74  19.49  19.24  19.05   None  80.67   
55  19.77  20.17  20.14  20.12  19.71  19.83  19.63  19.44  19.33   None   
44  19.34  19.73   19.7  19.68  19.68  19.36  19.53  19.39  19.28  19.22   
33   18.9  19.29  19.26  19.24  19.24  19.33  19.05  19.29  19.23  19.16   
22  18.45  1

In [10]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Renommer les DataFrames pour plus de cohérence
twosuits_df = equity_df.copy()
threesuits_df = equity_diff_color_df.copy()
foursuits_df = equity_diff_suit_df.copy()
